# Imports

In [1]:
%load_ext autoreload
%autoreload 2
import sys
from nlppen.extraccion.utils.Txt2Numbers import Txt2Numbers
from nlppen.analisis import Analisis
from nlppen.seleccion import Seleccion
from nlppen.spark_udfs import solo_portanto, solo_considerando, solo_resultando, solo_encabezado, spark_get_spacy
from nlppen.sentencias_estructurales import SentenciasEstructurales
from pyspark.sql import SparkSession
from pyspark.sql.functions import length

## Initialize spark context

In [2]:
spark = (SparkSession
         .builder
         .appName("Transforming sentences")
         .config("spark.num.executors", "8")
         .config("spark.executor.memory", "10g")
         .config("spark.executor.cores", "6")
         .config("spark.driver.memory", "10g")
         .config("spark.memory.offHeap.enabled", True)
         .config("spark.memory.offHeap.size", "25g")
         .config("spark.sql.execution.arrow.pyspark.enabled", "true")
         .getOrCreate())

sc = spark.sparkContext
sc.uiWebUrl

21/10/15 05:46:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


'http://7416269fd645:4040'

# Buscar terminos en la sección de por lo tanto de la sentencia y aplicar filtro del dataset

In [8]:
terminos = {
    'seguimiento': [r'\bseguimiento\b'],
    'se ordena': [r'\bse ordena\b', r'\bse le ordena\b', r'\bse les ordena\b'],
    'plan': [r'\bplan\b'],
    'plazo': [r'\bplazo\b']
}
seleccion = Seleccion(terminos, spark, parquet_path='./dataset', datasets_path='./datasets/estructurales')
print("Cantidad elementos originales : " + str(seleccion.sdf.count()))
seleccion.filtrar_sentencias(preprocess=solo_portanto, keepRowEmpty=True)
print("Cantidad elementos despues de filtrados : " + str(seleccion.sdf.count()))
estructurales = SentenciasEstructurales(seleccion)

Cantidad elementos originales : 1365


Cantidad elementos despues de filtrados : 1365


# Formar dataset de sentencias estructurales

In [9]:
from pyspark.sql.types import *

columnas = {
    'se ordena PER' : ArrayType(StringType()),
    'se ordena LOC' : ArrayType(StringType()),
    'se ordena ORG' : ArrayType(StringType()),
    'se ordena MISC' : ArrayType(StringType()),
    'se ordena GPE' : ArrayType(StringType()),
    'se ordena Ent Pub' : ArrayType(StringType())
}
estructurales.separarSeOrdena(columnas, True, True)

columnas = {
    'extension sentencia' : IntegerType(),
    'extension por lo tanto' : IntegerType()
}
estructurales.extraerExtension(columnas, True)

columnas = {
    'plazosDefinidos' : ArrayType(TimestampType())
}
estructurales.plazosDefinidos(columnas, True)

columnas = {
    'FechaSolicitud' : TimestampType(),
}

estructurales.extrarFechaRecibido(columnas, True)
columnas = {
    'num resolucion' : StringType()
}

estructurales.extraerNumeroSentencia(columnas, True)
#Sobreescribir el dataset de filtro de sentencias con las nuevas columnas
estructurales.seleccion.guardarDatos()

# Cargar procesamiento de sentencias estructurales.

In [12]:
terminos = {
    'seguimiento': [r'\bseguimiento\b'],
    'se ordena': [r'\bse ordena\b', r'\bse le ordena\b', r'\bse les ordena\b'],
    'plan': [r'\bplan\b'],
    'plazo': [r'\bplazo\b']
}
seleccion = Seleccion(terminos, spark, parquet_path='./dataset', datasets_path='./datasets/estructurales')
seleccion.cargarPreprocesados()

estructurales = SentenciasEstructurales(seleccion)
s = estructurales.seleccion.sdf
s.limit(10).toPandas()

/opt/conda/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:87: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: ArrayType(TimestampType,true)
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


,archivo,txt,tema,tema_prob,expediente,fechahora_ext,redactor_ext,tipoResolucion_ext,termino_ext,voto_salvado,...,se_ordena_LOC,se_ordena_ORG,se_ordena_MISC,se_ordena_GPE,se_ordena_Ent_Pub,extension_sentencia,extension_por_lo_tanto,plazosDefinidos,FechaSolicitud,num_resolucion
0,784F6.html,*050151950007CO* Exp: 05-015195-0007-CO Res. N...,PETICION,0.086251,291C05,2006-05-30 17:14:00,Armijo Sancho,Recurso de Amparo,Con lugar parcial,False,...,"[Circuito 02 de la, Colegio Líder INVU Las Cañas]","[Directora de la Escuela, Dirección Regional d...","[Director Regional de Educación de Alajuela, I]",None,"[Instituto Nacional de Vivienda y Urbanismo, I...",24112,2582,None,2005-11-24 10:05:00,2006-07804
1,72C2E.html,*060010870007CO* Exp: 06-001087-0007-CO Res. N...,TRABAJO,0.451749,060010870007CO,2006-02-22 14:44:00,Vargas Benavides,Recurso de Amparo,Con lugar,False,...,"[Ministerio de Educación Pública, Ministro de ...",None,None,None,[Ministerio de Educacion Publica],6022,1394,[1970-01-03 00:00:00],2006-01-31 12:27:00,2006-02200
2,7865A.html,*060032250007CO* Exp: 06-003225-0007-CO Res. N...,PENITENCIARIO,0.335066,060032250007CO,2006-07-19 19:41:00,Cruz Castro,Hábeas Corpus,Con lugar,False,...,None,None,None,None,None,4704,1297,None,2005-04-28 14:13:00,2006-010418
3,79A74.html,*060096310007CO* Exp: 06-009631-0007-CO Res. N...,PETICION,0.252051,060096310007CO,2006-08-23 16:26:00,Calzada Miranda,Recurso de Amparo,Con lugar,False,...,None,None,None,None,None,6041,457,None,2006-08-07 14:05:00,2006012365
4,72D53.html,*050165730007CO* Exp: 05-016573-0007-CO Res: 2...,TRABAJO,0.445111,050165730007CO,2006-03-10 13:10:00,Cruz Castro,Recurso de Amparo,Con lugar,False,...,None,None,None,None,None,4824,403,None,2005-12-23 14:20:00,2006-03283
5,7B98B.html,*060035870007CO* Exp: 06-003587-0007-CO Res. N...,PETICION,0.086251,060035870007CO,NaT,Sosto López,Recurso de Amparo,Con lugar,False,...,"[Ministra de Obras Publicas y Transportes, Min...",None,[Dependencias Técnicas],None,None,19759,1575,[1970-04-01 00:00:00],2006-03-28 09:41:00,2006010069
6,7DA84.html,*060142170007CO* Exp: 06-014217-0007-CO Res. N...,TRABAJO,0.644294,060142170007CO,2006-12-12 17:22:00,Jinesta Lobo,Recurso de Amparo,Con lugar,False,...,[Ministerio de Trabajo y Seguridad],[Director Nacional de Pensiones],None,None,None,14270,1466,None,2006-11-20 11:00:00,2006-017920
7,7544C.html,*060045980007CO* Exp: 06-004598-0007-CO Res. N...,TRABAJO,0.371132,060045980007CO,2006-05-16 15:54:00,Armijo Sancho,Recurso de Amparo,Con lugar,False,...,None,None,None,None,None,6579,401,None,2006-04-24 13:06:00,2006-006683
8,715B6.html,*050147230007CO* Exp: 05-014723-0007-CO Res. N...,PETICION,0.273704,050147230007CO,2006-02-10 11:51:00,Rodríguez Arroyo,Recurso de Amparo,Con lugar,False,...,[Alcaldesa Municipal de Montes de Oca],None,None,None,None,9301,1275,None,2005-11-15 08:52:00,2006-001463
9,6FF2A.html,*050143170007CO* Exp: 05-014317-0007-CO Res. N...,PENSION,0.680464,050143170007CO,2006-01-27 12:25:00,Molina Quesada,Recurso de Amparo,Con lugar,False,...,None,[Director Nacional de Pensiones],"[DNP-RE-2888-05, Ley de la Jurisdicción Consti...",None,None,6599,1453,[1970-06-30 00:00:00],2005-11-07 08:10:00,2006-00694


# Zona de trabajo (ignorar)

In [14]:
s = estructurales.seleccion.sdf
print("Totales :", str(s.count()))
print("Nulos : ", str(s.filter(s.num_resolucion.isNull()).count()))
print("Completos : ", str(s.filter(s.num_resolucion.isNotNull()).count()))
print("Accuracy : " ,s.filter(s.num_resolucion.isNotNull()).count()/s.count())

s = s.filter(s.num_resolucion.isNull())
#s = estructurales.seleccion.sdf

Totales : 1365
Nulos :  3
Completos :  1362
Accuracy :  0.9978021978021978


In [16]:
s.filter(s.num_resolucion.isNull()).groupby(s.anno).count().toPandas().sort_values('anno')

,anno,count
2,1993,1
0,1997,1
1,2019,1


In [41]:
s.filter(s.num_resolucion.isNull()).groupby(s.anno).count().toPandas().sort_values('anno')

,anno,count
2,1993,1
0,1997,1
1,2019,1
